In [ ]:
!pip install lightgbm

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install scikit-learn

In [ ]:
!pip install sklearn_relief

In [ ]:
# Import des bibliothèques nécessaires
from scipy import interp  # Pour l'interpolation
from sklearn.metrics import (  # Pour diverses métriques d'évaluation
    roc_curve, auc, confusion_matrix, precision_score, recall_score, f1_score,
    accuracy_score, roc_auc_score
)
from sklearn.linear_model import ElasticNet, LogisticRegression, RidgeClassifier, SGDClassifier  # Différents classificateurs linéaires
from sklearn.preprocessing import MinMaxScaler  # Pour la mise à l'échelle des caractéristiques
from sklearn.tree import DecisionTreeClassifier  # Classificateur Arbre de Décision
from sklearn.svm import SVC  # Classificateur Vecteur de Support
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  # Classificateur Analyse Discriminante Linéaire
from sklearn.ensemble import (  # Classificateurs ensemblistes
    RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier,
    BaggingClassifier, VotingClassifier
)
from sklearn.naive_bayes import GaussianNB, MultinomialNB  # Classificateurs Naïve Bayes
from sklearn.neighbors import KNeighborsClassifier  # Classificateur K plus proches voisins
from sklearn.metrics import confusion_matrix  # Calcul de la matrice de confusion
from sklearn.model_selection import (  # Pour la validation croisée et le fractionnement train-test
    train_test_split, cross_val_score, StratifiedKFold, cross_val_predict
)
from sklearn.feature_selection import (  # Méthodes de sélection de caractéristiques
    SelectKBest, chi2, f_classif, mutual_info_classif, SelectFromModel
)
import lightgbm as lgb  # Classificateur LightGBM
import catboost as cb  # Classificateur CatBoost
from xgboost import XGBClassifier  # Classificateur XGBoost
import sklearn_relief as sr  # Sélection de caractéristiques Relief
from sklearn.inspection import permutation_importance  # Calcul de l'importance de la permutation
import joblib  # Pour sauvegarder les modèles
import pandas as pd  # Bibliothèque de manipulation de données
import numpy as np  # Bibliothèque de calcul numérique
import matplotlib.pyplot as plt  # Bibliothèque de traçage
import time  # Pour la mesure du temps
import warnings  # Pour supprimer les avertissements

warnings.filterwarnings("ignore")  # Ignorer les avertissements

# Charger l'ensemble de données
dataset = pd.read_csv("/content/genetic-AD-NL-2.csv", index_col=('Id-patient'))
classes = dataset['final_diagnosis'].map({'NL': 1, 'AD': 0})  # Mapper les classes NL et AD à 1 et 0
data = dataset.iloc[:, 2:]  # Extraire les caractéristiques de l'ensemble de données

# Mise à l'échelle des caractéristiques
scaler = MinMaxScaler(feature_range=(0, 1))  # Initialiser MinMaxScaler
data_scaled = scaler.fit_transform(data)  # Mettre à l'échelle les caractéristiques
data_scaled_df = pd.DataFrame(data_scaled, columns=data.columns, index=data.index)  # Convertir les données mises à l'échelle en DataFrame

# Définir les classificateurs
lr = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)  # Classificateur Régression Logistique avec pénalité elasticnet
lda = LinearDiscriminantAnalysis()  # Classificateur Analyse Discriminante Linéaire
dtc = DecisionTreeClassifier()  # Classificateur Arbre de Décision
svc = SVC(kernel='linear', C=1)  # Classificateur Vecteur de Support avec noyau linéaire
rfc = RandomForestClassifier(max_depth=1000, n_estimators=1000, random_state=42)  # Classificateur Forêt Aléatoire
rc = RidgeClassifier()  # Classificateur Ridge
sgd = SGDClassifier()  # Classificateur Descente de Gradient Stochastique
grb = GradientBoostingClassifier()  # Classificateur Gradient Boosting
etc = ExtraTreesClassifier()  # Classificateur Extra Trees
xgboost = XGBClassifier()  # Classificateur XGBoost

# Fonction pour la validation croisée et l'évaluation des performances
def cross(estimator, x):
    scoring = 'accuracy'  # Métrique de scoring pour l'évaluation
    models = [('LogisticRegression', lr), ('SVM', svc), ('RidgeClassifierD', rc), ('SGDClassifier', sgd), ('GaussianNB', GaussianNB())]  # Liste des classificateurs avec leurs noms
    results = []  # Liste pour stocker les résultats de la validation croisée
    names = []  # Liste pour stocker les noms des classificateurs

    # Sélection de caractéristiques
    sfm = SelectKBest(score_func=f_classif, k=round(x))  # Sélection de caractéristiques en utilisant la valeur F ANOVA
    sfm.fit(data_scaled_df, classes)  # Adapter le modèle de sélection de caractéristiques
    X_important_train = sfm.transform(data_scaled_df)  # Transformer les données avec les caractéristiques sélectionnées

    for name, model in models:
        kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)  # Séparateur de validation croisée Stratified K-Folds
        start_time = time.time()  # Temps de début pour mesurer le temps d'exécution
        cv_results = cross_val_score(model, X_important_train, classes, cv=kfold, scoring=scoring).mean()  # Validation croisée
        results.append(cv_results)  # Ajouter le résultat de la validation croisée
        names.append(name)  # Ajouter le nom du classificateur

        # Métriques de performance
        y_pred = cross_val_predict(model, X_important_train, classes, cv=kfold)  # Étiquettes prédites par validation croisée
        tn, fp, fn, tp = confusion_matrix(classes, y_pred).ravel()  # Matrice de confusion
        specificity = tn / (tn + fp)  # Calcul de la spécificité
        sensitivity = tp / (tp + fn)  # Calcul de la sensibilité
        aucc = roc_auc_score(classes, y_pred)  # Surface sous la courbe ROC

        # Afficher les métriques de performance
        print(f"{name}\nPrécision: {100*cv_results:.2f}% ({cv_results.std():.2f})")
        print(f"Précision: {precision_score(classes, y_pred)*100:.2f}%")
        print(f"Spécificité: {specificity*100:.2f}%")
        print(f"Rappel/Sensibilité: {sensitivity*100:.2f}%")
        print(f"Score F1: {f1_score(classes, y_pred)*100:.2f}%")
        print(f"AUC: {aucc*100:.2f}%")
        print(f"--- {time.time() - start_time:.2f} secondes ---\n")  # Afficher le temps d'exécution

# Mesurer le temps d'exécution de la validation croisée
start_time = time.time()
cross(lr, 100)
print(f"--- {time.time() - start_time:.2f} Temps-de-toutes-les-opérations ---")  # Afficher le temps d'exécution total


LogisticRegression
Accuracy: 81.84% (0.00)
Precision: 81.41%
Specificity: 53.97%
Recall/Sensitivity: 94.78%
F1 Score: 87.59%
AUC: 74.37%
--- 0.69 seconds ---

SVM
Accuracy: 79.13% (0.00)
Precision: 84.40%
Specificity: 65.08%
Recall/Sensitivity: 88.81%
F1 Score: 86.55%
AUC: 76.94%
--- 0.08 seconds ---

RidgeClassifierD
Accuracy: 82.21% (0.00)
Precision: 84.89%
Specificity: 66.67%
Recall/Sensitivity: 88.06%
F1 Score: 86.45%
AUC: 77.36%
--- 0.30 seconds ---

SGDClassifier
Accuracy: 79.66% (0.00)
Precision: 85.59%
Specificity: 73.02%
Recall/Sensitivity: 75.37%
F1 Score: 80.16%
AUC: 74.19%
--- 0.14 seconds ---

GaussianNB
Accuracy: 68.03% (0.00)
Precision: 81.48%
Specificity: 68.25%
Recall/Sensitivity: 65.67%
F1 Score: 72.73%
AUC: 66.96%
--- 0.05 seconds ---

--- 2.24 Time-of-all-operation ---


# New Section